In [1]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

import os,sys

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ActionChains


import time


def flushPrint(s):
    sys.stdout.write('\r')
    sys.stdout.write('%s' % s)
    sys.stdout.flush()



In [2]:
df = pd.read_pickle('./info-jstor.pkl')
#df = pd.read_csv('./result-jstor.csv')

In [3]:
#df['Downloaded'] = pd.Series([0 for i in range(len(df))])

In [4]:
df

,Author,Title,Year,Doi,Downloaded
0,Dwight B. Billings and Shaunna L. Scott,Religion and Political Legitimation,1994,10.2307/2083364,1
1,Harold W. Pfautz,The Sociology of Secularization: Religious Groups,1955,10.2307/2771726,1
2,Nancy Theberge,A Critique of Critiques: Radical and Feminist ...,1981,10.2307/2578439,1
3,Frederick C. Engelmann and Mildred A. Schwartz,Partisan Stability and the Continuity of a Seg...,1974,10.2307/2776351,1
4,Wade Clark Roof,Traditional Religion in Contemporary Society: ...,1976,10.2307/2094469,1
...,...,...,...,...,...
57,Steve Bruce,The Pervasive World-View: Religion in Pre-Mode...,1997,10.2307/591602,0
58,Kenneth Thompson,Religion: The British Contribution,1990,10.2307/590666,0
59,Ariana Need and Nan Dirk de Graaf,'Losing My Religion': A Dynamic Analysis of Le...,1996,10.2307/522776,0
60,John Seidler,Contested Accommodation: The Catholic Church a...,1986,10.2307/2578784,0


In [5]:
def mark_download(num_1,df):
    for i in num_1:
        df.loc[i, 'Downloaded']=1
        df.to_csv("result-jstor.csv", index_label="index_label")
        df.to_pickle('./info-jstor.pkl')
    return df

In [6]:
num_1 = [0,1,2,3,4,5,6,7,8,10,11,13,14,15,16,17,18,22,23,25,39,40]
#mark_download(num_1,df)

In [7]:
#设定下载地址
options=webdriver.ChromeOptions()
out_path='/Users/yuchanghong/Downloads'
prefs={'profile.default_content_settings.popups': 0, 'download.default_directory': out_path}
options.add_experimental_option('prefs', prefs)

In [8]:
#为了帮助判断是否下载成功，定义一个计算文件夹中的文献数的函数
def count_paper(path):
    files = os.listdir(path)
    file_num=len(files)
    return (file_num)

In [9]:
c = count_paper('/Users/yuchanghong/Downloads')
c

4

In [10]:
#定义一个函数，帮助找到最新下载的一个文件
def find_new_file(path):
    file_lists=os.listdir(path)
    file_lists.remove(".DS_Store")
    file_lists.sort(key=lambda fn: os.path.getmtime(path + "/" + fn)
                    if not os.path.isdir(path + "/" + fn) else 0)
    file=os.path.join(path, file_lists[-1])
    return file

In [ ]:
#根据已经获取的doi在sci-hub上批量下载文献

label=True #作用是当确定所有文献都已经下载完之后，就使循环停止运行
times=0 #记录整体循环的次数；有些没有一次就下载成功的文章需要多次尝试，通过给times设定上限确保程序最终会停止
r = 9

while label==True and times<5:
    r = input("0读取，其他不读取：")
    if r==0:
        df = pd.read_pickle('./info-jstor'+str(times)+'.pkl')
    times+=1
    print("循环次数：",times)
    unsuc=0 #判断是否有未下载成功的文献
    
    for i in range(len(df)):
    #for i in range(1):
        flushPrint(i+1)
        count=count_paper(out_path)
       
        if df.iloc[i,4]==0:
            
            try:
                driver=webdriver.Chrome(executable_path="/Users/yuchanghong/opt/anaconda3/chromedriver",chrome_options=options)
                link="https://sci-hub.se/"+df.iloc[i,3]
                driver.get(link)
                soup=BeautifulSoup(driver.page_source,'lxml')
                temp=str(soup.select("#buttons>button")).split("href='")[1].split("'\">")
                download_link='http://'+temp[0].lstrip('/')
                #print(download_link)
                driver.get(download_link)
                time.sleep(30)
                print('现有文件数',count_paper(out_path))
                
                #判断文献是否下载成功
                if count_paper(out_path)==count+1:
                    df.iloc[i,4]=1
                    print('rename')
                    #给新下载的文件重命名
                    new_paper=find_new_file(out_path)
                    new_name=str(df.iloc[i,0].split(',')[0])+"_"+df.iloc[i,2]
                    os.rename(new_paper,out_path+"/"+new_name+".pdf")
    
                
            except:
                unsuc+=1
            
            driver.quit()
    if unsuc==0:
        label=False
    df.to_csv("result-jstor.csv", index_label="index_label")
    df.to_pickle('./info-jstor'+str(times)+'.pkl')

0读取，其他不读取： 8


循环次数： 1
10

/var/folders/r7/p1s2dy_d55zbvx8s9j4nd_x00000gn/T/ipykernel_1093/3794922637.py:23: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(executable_path="/Users/yuchanghong/opt/anaconda3/chromedriver",chrome_options=options)
/var/folders/r7/p1s2dy_d55zbvx8s9j4nd_x00000gn/T/ipykernel_1093/3794922637.py:23: DeprecationWarning: use options instead of chrome_options
  driver=webdriver.Chrome(executable_path="/Users/yuchanghong/opt/anaconda3/chromedriver",chrome_options=options)


现有文件数 5
rename
13现有文件数 6
rename
20现有文件数 7
rename
21现有文件数 8
rename
29现有文件数 9
rename
33现有文件数 10
rename
38现有文件数 11
rename
39现有文件数 12
rename
42现有文件数 13
rename
43现有文件数 14
rename
44现有文件数 15
rename
45现有文件数 16
rename
46现有文件数 17
rename
47现有文件数 18
rename
48现有文件数 19
rename
49现有文件数 20
rename
50现有文件数 21
rename
51现有文件数 22
rename
52现有文件数 23
rename
62现有文件数 24
rename


0读取，其他不读取： 9


循环次数： 2
54现有文件数 25
rename
55现有文件数 26
rename
56现有文件数 27
rename
57现有文件数 28
rename
58现有文件数 29
rename
59现有文件数 30
rename
60现有文件数 31
rename
61现有文件数 32
rename
62

0读取，其他不读取： 0


循环次数： 3
53现有文件数 33
rename
62

0读取，其他不读取： 0


循环次数： 4
32

In [12]:
sorted(list(df['Author']))

['',
 '',
 '',
 'Andrew J. Weigert and Darwin L. Thomas',
 'Ariana Need and Nan Dirk de Graaf',
 'Armand L. Mauss',
 'DAMON MAYRL',
 'DAMON MAYRL',
 'David Martin',
 'David Voas and Alasdair Crockett and Daniel V. A. Olson',
 'David Voas and Mark Chaves',
 'Dwight B. Billings and Shaunna L. Scott',
 'Edward A. Tiryakian',
 'Elaine Howard Ecklund and Jerry Z. Park and Phil Todd Veliz',
 'Eviatar Zerubavel',
 'Fenggang Yang and Helen Rose Ebaugh',
 'Frank J. Lechner',
 'Frank J. Lechner',
 'Frank van\xa0Tubergen and Manfred te\xa0Grotenhuis and Wout Ultee',
 'Frederick C. Engelmann and Mildred A. Schwartz',
 'Gerald Marwell and N. J. Demerath',
 'Guenther Roth',
 'Harold W. Pfautz',
 'Harold W. Pfautz',
 'Heiner Meulemann',
 'Heiner Meulemann',
 'Jacqueline Scott',
 'Jeffrey K. Hadden',
 'John Seidler',
 'Kenneth Thompson',
 'Kenneth Thompson and Anita Sharma',
 'Kenneth Westhues',
 'LARS BO KASPERSEN and JOHANNES LINDVALL',
 'Larry R. Petersen and Gregory V. Donnenwerth',
 'Liana Giorgi

In [2]:
#df = pd.read_pickle('./info-jstor'+str(0)+'.pkl')

In [7]:
#df = pd.read_pickle('./info-jstor.pkl')

In [12]:
df.to_csv("result-jstor.csv", index_label="index_label")

In [13]:
df.to_pickle('./info-jstor.pkl')